# Movie Recommendation (Boltzmann Machine)

## 1. Dataset

This dataset is from grouplen.org. url:https://grouplens.org/datasets/movielens/

We take 100k and 1m dataset for the training dataset. 

## 2.Model

We are going to use Restricted Boltzmann Machine (RBM) to generate a recommend system for movies. We implement the contrastive divergence for this model.

## 3. Data Preprocessing

### Import relevant libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Import dataset

In [2]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header= None, engine='python', encoding='latin-1')

In [3]:
movies

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [4]:
users = pd.read_csv('ml-1m/users.dat', sep='::', header= None, engine='python', encoding='latin-1')

In [5]:
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [6]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header= None, engine='python', encoding='latin-1')

In [7]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


### Preparing the training set and the test set

In [8]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')

In [9]:
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [10]:
training_set = np.array(training_set, dtype='int')

In [11]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [12]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')

In [13]:
test_set = np.array(test_set, dtype='int')

In [14]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

### Getting the number of users and movies

In [15]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [16]:
nb_users

943

In [17]:
nb_movies

1682

### Converting the data into an array with users in lines and movies in columns

In [18]:
# Create a function to do this
def convert(data):
    """
    Create an array to show user ID in line and movies in columns.
    """
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0]==id_users]
        id_ratings = data[:,2][data[:,0]==id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [19]:
training_set = convert(training_set)

In [20]:
test_set = convert(test_set)

In [21]:
len(training_set)

943

In [22]:
len(test_set)

943

### Converting the data into Torch tensors

In [23]:
training_set = torch.FloatTensor(training_set)

In [24]:
test_set = torch.FloatTensor(test_set)

In [25]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [26]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

### Converting the rating into binary ratings 1 (Liked) or 0 (Not Liked)

In [27]:
# Replace the unrated movies to -1
training_set[training_set == 0] = -1

# We considered that 1 star or 2 stars ratings refer to not like the movie
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0

# Turn 3,4,5 star ratings to be like the movie
training_set[training_set >= 3] = 1

In [28]:
# Replace the unrated movies to -1
test_set[test_set == 0] = -1

# We considered that 1 star or 2 stars ratings refer to not like the movie
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0

# Turn 3,4,5 star ratings to be like the movie
test_set[test_set >= 3] = 1

In [29]:
training_set

tensor([[-1.,  1.,  1.,  ..., -1., -1., -1.],
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.]])

In [30]:
test_set

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

## 4. Modelling

### Create the architecture of the Neural Network

Since we will create a class for the RBM model, we need to make 3 functions to build our RBM model:
1. initialize the RBM.
2. sample H: That will sample the probabilities of the hidden nodes given the visible nodes.
3. sample V: That will sample the probabilities of the visible nodes given the hidden nodes.
4. contrastive divergence: to approximate the likelihood gradient.

**Why need sample H function?:**

Use Gibbs sampling to approximate the log likelihood gradients. And to apply Gibbs sampling, we need to compute the probabilities of the hidden nodes given the visible nodes. Once we have this probabilities, we can sample the activation functions of the hidden nodes.

**Why need sample V function?:**

From the values in the hidden nodes, that is whether thsy were activated or not, we will also estimate the probabilities of the visible nodes, that is the probabilities that each of the visible node equals one.

We need it for Gibbs sampling that we will apply next when we approximate the log likelihood gradient.

**Contrastive Divergence**:

We also need to create this function. RBM is an energy-based model, and this is the energy function we are trying to minimized. This energy function depends on the weight of the model, that is, all the weights in this tensor of weights that we defined previously needed to optimized in order to minimize the energy. It can also be seen as a probability graphical model. The goal is to maximum the log-likelihood, so we need to compute the gradient.

In [46]:
class RBM():
    def __init__(self, nv, nh):
        self.weights = torch.randn(nh, nv)
        self.hidden_bias = torch.randn(1, nh) # bias for the hidden nodes
        self.visible_bias = torch.randn(1, nv) # bias for the visible nodes
    
    def sample_h(self, x): # x is the visible neurons, v in the probabilities, p(h/v)
        wx = torch.mm(x, self.weights.t())
        activation = wx + self.hidden_bias.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    
    def sample_v(self, y): # y is the hidden neurons, h in the probabilities, p(v/h)
        # We don't take tranpose cause we were computing p_h_given_v since w is the weight matrix of p_v_given_h.
        wy = torch.mm(y, self.weights)
        activation = wy + self.visible_bias.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    
    def train(self, v0, vk, ph0, phk):
        # vk: the visible nodes obtained after K samplings(K iterations and K contrastive divergence).
        # ph0: the vector of probabilities that at the first iteration the hidden nodes equal one given the values of v0
        # phk: the probabilities of the hideen nodes after K sampling given the values of the visible nodes, vk.
        
        self.weights += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.visible_bias += torch.sum((v0 - vk),0)
        self.hidden_bias += torch.sum((ph0 - phk),0)

### Create an RBM object

In [47]:
# number of visible nodes correspond to the number of movies
nv = len(training_set[0])
nv

1682

In [48]:
# number of hidden nodes correspond to the number of features we want to detect, we can initialize it with 100.
nh = 100

In [49]:
batch_size = 100

In [50]:
# Create the RBM model
rbm = RBM(nv, nh)

### Train the RBM

In [51]:
# Choose an epoch
nb_epoch = 10

In [52]:
# Start training

for epoch in range(1, nb_epoch+1):
    # measure the loss between real ratings and predicted ratings
    # We introduce the absolute distance as the loss function
    train_loss = 0
    
    # counter
    s = 0.
    
    for id_user in range(0, nb_users - batch_size, batch_size):
        # vk: the output of Gibbs sampling after the K steps of the random walk. at the start it's actually the input batch of observations
        vk = training_set[id_user:id_user + batch_size]
        # v0: the ratings of the movies that were already rated by the 100 users in this batch
        v0 = training_set[id_user:id_user + batch_size]
        # ph0: the probabilities that the hidden node at the start equal one given the real ratings
        ph0,_ = rbm.sample_h(v0)
        
        # for loop for the K steps of contrastive divergence
        # Gibbs sampling, is an MCMC (Markov Chain Monte Carlo) technique
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0] # We freeze out the ratings that are not rated since these cannot be updated
        phk,_ = rbm.sample_h(vk)
        
        # Apply the train function
        rbm.train(v0, vk, ph0, phk)
        
        # Update the train loss
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        
        # Update the counter
        s += 1
        
    # print what happen in the function
    print('epoch: ' + str(epoch) + ' loss: '+ str(train_loss/s))
        

epoch: 1 loss: tensor(0.3443)
epoch: 2 loss: tensor(0.2568)
epoch: 3 loss: tensor(0.2497)
epoch: 4 loss: tensor(0.2484)
epoch: 5 loss: tensor(0.2493)
epoch: 6 loss: tensor(0.2377)
epoch: 7 loss: tensor(0.2469)
epoch: 8 loss: tensor(0.2457)
epoch: 9 loss: tensor(0.2465)
epoch: 10 loss: tensor(0.2458)


## 5. Evaluation

### Test the RBM

In [53]:
# Testing the RBM

test_loss = 0

# counter
s = 0.

for id_user in range(nb_users):
    # We are using the inputs of the training set (v) to get the neurons of hte RBM to get the predicted ratings of the test set.
    v = training_set[id_user:id_user + 1]
    # vt: the target rating
    vt = test_set[id_user:id_user + 1]

    # We just need one step sampling for testing
    if len(vt[vt>=0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)

        # Update the test loss
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))

        # Update the counter
        s += 1.

# print what happen in the function
print('test loss: '+ str(test_loss/s))

test loss: tensor(0.2604)


In the end, we use Average Absolute distance to evaluate the RBM model. There is another way to measure the loss and that is RMSE (Root_Mean_Squared_Error). Both works and give the accuracy rate about 75%.